# Hybrid Streaming Optimizer API Demo

This notebook demonstrates how to use `method='hybrid_streaming'` with both
`curve_fit()` and `curve_fit_large()` functions.

The hybrid streaming optimizer provides:
- Parameter normalization for better gradient signals
- Adam warmup for robust initial convergence
- Streaming Gauss-Newton for exact covariance computation
- Automatic memory management for large datasets

In [1]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [2]:
import numpy as np
import jax.numpy as jnp
from nlsq import curve_fit, curve_fit_large

In [3]:
def exponential_decay(x, a, b, c):
    """Three-parameter exponential decay model."""
    return a * jnp.exp(-b * x) + c

In [4]:
# Generate synthetic data
np.random.seed(42)
x = np.linspace(0, 10, 2000)
true_params = np.array([5.0, 0.5, 1.0])
y_true = exponential_decay(x, *true_params)
y = y_true + np.random.normal(0, 0.1, len(x))

print(f"Dataset: {len(x)} samples")
print(f"True parameters: a={true_params[0]}, b={true_params[1]}, c={true_params[2]}")

Dataset: 2000 samples
True parameters: a=5.0, b=0.5, c=1.0


## Example 1: Basic usage with curve_fit()

In [5]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=1,
)

# Unpack result
popt, pcov = result

print(f"\nFitted parameters: {popt}")
print(f"True parameters:   {true_params}")
print(f"Parameter errors:  {np.abs(popt - true_params)}")
print(f"\nCovariance matrix diagonal: {np.diag(pcov)}")
print(f"Parameter std errors: {np.sqrt(np.diag(pcov))}")

Adaptive Hybrid Streaming Optimizer
Dataset size: 2,000 points
Parameters: 3
Normalization: auto
Precision: auto



Phase 0: Normalization setup complete (0.237s)
  Strategy: p0
  Precision: <class 'jax.numpy.float32'>

Phase 1: Adam warmup...
  Precision: <class 'jax.numpy.float32'>


Phase 1 complete: 500 iterations (2.271s)
  Best loss: 1.311939e-02
  Switch reason: Maximum iterations reached

Phase 2: Streaming Gauss-Newton...
  Precision: <class 'jax.numpy.float64'>


Phase 2 complete: 100 iterations (5.043s)
  Final cost: inf
  Convergence: Maximum iterations reached
  Gradient norm: 4.738567e+01

Phase 3: Computing covariance...
  Precision: <class 'jax.numpy.float64'>


Phase 3 complete (0.288s)
  Residual variance (σ²): 1.312076e-02

Optimization Complete
Total time: 7.840s
Final parameters: [4.85521276 0.44692999 0.91296743]
Parameter std errors: [0.01088332 0.0023364  0.00563285]

Fitted parameters: [4.85521276 0.44692999 0.91296743]
True parameters:   [5.  0.5 1. ]
Parameter errors:  [0.14478724 0.05307001 0.08703257]

Covariance matrix diagonal: [1.18446665e-04 5.45878244e-06 3.17289643e-05]
Parameter std errors: [0.01088332 0.0023364  0.00563285]


/home/wei/Documents/GitHub/NLSQ/nlsq/minpack.py:1517: RuntimeWarning: invalid value encountered in multiply
  result["pcov"] = result_dict.get("pcov", np.eye(n) * np.inf)


## Example 2: With parameter bounds

In [6]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    bounds=([0, 0, 0], [10, 2, 5]),
    method='hybrid_streaming',
    verbose=0,  # Silent mode
)

popt, pcov = result
print(f"Fitted parameters (bounded): {popt}")
print(f"Within bounds: {np.all(popt >= [0, 0, 0]) and np.all(popt <= [10, 2, 5])}")

Fitted parameters (bounded): [4.96980627 0.49540144 1.00113319]
Within bounds: True


## Example 3: Large dataset with curve_fit_large()

In [7]:
# Generate larger dataset
x_large = np.linspace(0, 10, 10000)
y_large = exponential_decay(x_large, *true_params) + np.random.normal(0, 0.1, len(x_large))

popt, pcov = curve_fit_large(
    exponential_decay,
    x_large,
    y_large,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=1,
)

print(f"\nFitted parameters (large dataset): {popt}")
print(f"True parameters:                   {true_params}")
print(f"Parameter errors:                  {np.abs(popt - true_params)}")

Adaptive Hybrid Streaming Optimizer
Dataset size: 10,000 points
Parameters: 3
Normalization: auto
Precision: auto

Phase 0: Normalization setup complete (0.001s)
  Strategy: p0
  Precision: <class 'jax.numpy.float32'>

Phase 1: Adam warmup...
  Precision: <class 'jax.numpy.float32'>


Phase 1 complete: 500 iterations (1.691s)
  Best loss: 1.322629e-02
  Switch reason: Maximum iterations reached

Phase 2: Streaming Gauss-Newton...
  Precision: <class 'jax.numpy.float64'>


Phase 2 complete: 100 iterations (4.164s)
  Final cost: inf
  Convergence: Maximum iterations reached
  Gradient norm: 2.353244e+02

Phase 3: Computing covariance...
  Precision: <class 'jax.numpy.float64'>
Phase 3 complete (0.003s)
  Residual variance (σ²): 1.321226e-02

Optimization Complete
Total time: 5.860s
Final parameters: [4.86882487 0.44988159 0.91113714]
Parameter std errors: [0.00490545 0.00105154 0.00251071]

Fitted parameters (large dataset): [4.86882487 0.44988159 0.91113714]
True parameters:                   [5.  0.5 1. ]
Parameter errors:                  [0.13117513 0.05011841 0.08886286]


/home/wei/Documents/GitHub/NLSQ/nlsq/__init__.py:463: RuntimeWarning: invalid value encountered in multiply
  pcov = result_dict.get('pcov', np.eye(len(p0)) * np.inf)


## Example 4: Custom configuration via kwargs

In [8]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=0,
    # HybridStreamingConfig overrides:
    warmup_iterations=300,
    normalization_strategy='p0',  # 'bounds' requires explicit bounds
    phase2_max_iterations=100,
)

popt, pcov = result
print(f"Fitted parameters (custom config): {popt}")
print(f"Result attributes available: {list(result.keys())[:10]}")

Fitted parameters (custom config): [4.85521276 0.44692999 0.91296743]
Result attributes available: ['x', 'success', 'message', 'fun', 'pcov', 'perr', 'streaming_diagnostics', '_predictions_cache', '_residuals_cache', 'model']


## Example 5: Accessing full result details

In [9]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=0,
)

print(f"Success: {result['success']}")
print(f"Message: {result['message']}")
print(f"Final cost: {result.get('cost', 'N/A')}")

if 'streaming_diagnostics' in result:
    diag = result['streaming_diagnostics']
    print(f"\nStreaming diagnostics available:")
    print(f"  Keys: {list(diag.keys())}")

Success: True
Message: Maximum iterations reached
Final cost: N/A

Streaming diagnostics available:
  Keys: ['phase_timings', 'phase_iterations', 'total_time', 'warmup_diagnostics', 'gauss_newton_diagnostics', 'phase_history']


## Summary

The hybrid streaming optimizer combines:
- **Parameter normalization** for better gradient signals when parameters have different scales
- **Adam warmup** for robust initial convergence from poor initial guesses
- **Streaming Gauss-Newton** for fast second-order convergence near the optimum
- **Exact covariance computation** for reliable uncertainty estimates

Use `method='hybrid_streaming'` when:
- Parameters span many orders of magnitude
- Large datasets (100K+ points) with memory constraints
- Need production-quality uncertainty estimates
- Standard optimizers converge slowly